##Add coordinates using geopy

In [56]:
import pandas as pd
from geopy import geocoders
from geopandas.geocode import geocode
from geopy.geocoders import Nominatim

In [24]:
df_ebola = pd.read_csv("csv/out/ebola-outbreaks-before-2014-country-codes.csv", encoding="utf-8", index_col=False)

df_eb = df_ebola.drop(df_ebola.columns[[0, 1]], axis=1) # remove 2 useless colums of indexes

df_eb.head()

,Start_Datetime,Start date,End_Datetime,End date,Duration (days),Year(s),Country code (ISO 2 digits),Country,Ebola subtype,Reported number of human cases,Reported number of deaths among cases,Reported % of deaths among cases
0,2012-11-01 00:00:00,2012-11-01,2013-01-31 00:00:00,2013-01-31,91,November 2012-January 2013,UG,Uganda,Sudan virus,6,3,0.500
1,2012-06-01 00:00:00,2012-06-01,2012-11-30 00:00:00,2012-11-30,182,June-November 2012,CD,Democratic Republic of the Congo,Bundibugyo virus,36,13,0.361
2,2012-06-01 00:00:00,2012-06-01,2012-10-31 00:00:00,2012-10-31,152,June-October 2012,UG,Uganda,Sudan virus,11,4,0.364
3,2011-05-01 00:00:00,2011-05-01,2011-05-01 00:00:00,2011-05-01,0,2011-05-01,UG,Uganda,Sudan virus,1,1,1.000
4,2008-12-01 00:00:00,2008-12-01,2009-02-28 00:00:00,2009-02-28,89,December 2008-February 2009,CD,Democratic Republic of the Congo,Zaire virus,32,15,0.470


In [83]:
# Use the geocode function from GeoPandas and bypass the error with MapQuest

countries = set(list(df_eb["Country"]))

list_countries = list(countries)

print len(list_countries)
print list_countries

15
[u"C\xf4te d'Ivoire (Ivory Coast)", u'Italy', u'USA', u'South Africa', u'Democratic Republic of the Congo (formerly Zaire)', u'Philippines', u'Republic of the Congo', u'Democratic Republic of the Congo', u'Gabon', u'Sudan (South Sudan)', u'Uganda', u'Zaire (Democratic Republic of the Congo - DRC)', u'England', u'Zaire', u'Russia']


In [49]:
geocoders.MapQuest = None

try:
    geocodes = geocode(list_countries)
except:
    print "Can't find location!"
    pass

geocodes

Can't find location!


,address,geometry
0,Uganda,POINT (32.290275 1.373333)
1,Democratic Republic of the Congo,POINT (21.758664 -4.038333)
2,Uganda,POINT (32.290275 1.373333)
3,Uganda,POINT (32.290275 1.373333)
4,Democratic Republic of the Congo,POINT (21.758664 -4.038333)
5,Philippines,POINT (121.774017 12.879721)
6,Uganda,POINT (32.290275 1.373333)
7,Democratic Republic of the Congo,POINT (21.758664 -4.038333)
8,Russia,POINT (105.318756 61.52401)
9,Sudan,POINT (30.217636 12.862807)


In [55]:
type(geocodes["geometry"][0])

shapely.geometry.point.Point

__=> It didn't work for some countries. Also I can't have latitudes and longitudes in separate columns.__

In [107]:
cc = df_eb[["Country code (ISO 2 digits)", "Country"]]
print len(cc)
print cc[:5]
print("\n")
print cc[0:1]["Country"][0]
print cc[1:2]["Country"][1]
print cc[2:3]["Country"][2]

33
  Country code (ISO 2 digits)                           Country
0                          UG                            Uganda
1                          CD  Democratic Republic of the Congo
2                          UG                            Uganda
3                          UG                            Uganda
4                          CD  Democratic Republic of the Congo


Uganda
Democratic Republic of the Congo
Uganda


In [117]:
geolocator = Nominatim()

all_locations = {"codes": [], "coordinates": []}

for i in range(len(cc)):
    iso_code = cc[i:i+1]["Country code (ISO 2 digits)"][i]
    country = cc[i:i+1]["Country"][i]
    print i, country, iso_code
    
    if iso_code in all_locations["codes"]:
        all_locations["codes"].append(iso_codes)
        coord = all_locations["coordinates"][all_locations["codes"].index(iso_code)]
        all_locations["coordinates"].append(coord)
        
    else:
        all_locations["codes"].append(iso_code)
        try:
            location = geolocator.geocode(country)
            lat, lon = location.latitude, location.longitude
            all_locations["coordinates"].append((lat, lon))
    
        except AttributeError:
            print "attribute error!"
            location = geolocator.geocode(iso_code)
            lat, lon = location.latitude, location.longitude
            all_locations["coordinates"].append((lat, lon))
        except:
            print "No idea what went wrong here!"
            all_locations["coordinates"].append((0, 0))

0 Uganda UG
1 Democratic Republic of the Congo CD
2 Uganda UG
3 Uganda UG
4 Democratic Republic of the Congo CD
5 Philippines PH
6 Uganda UG
No idea what went wrong here!
7 Democratic Republic of the Congo CD
8 Russia RU
9 Sudan (South Sudan) SD
10 Republic of the Congo CG
11 Republic of the Congo CG
12 Republic of the Congo CG
13 Gabon GA
14 Uganda UG
15 Russia RU
No idea what went wrong here!
16 Philippines PH
No idea what went wrong here!
17 USA US
No idea what went wrong here!
18 South Africa ZA
No idea what went wrong here!
19 Gabon GA
No idea what went wrong here!
20 Gabon GA
No idea what went wrong here!
21 Democratic Republic of the Congo (formerly Zaire) CD
No idea what went wrong here!
22 Côte d'Ivoire (Ivory Coast) CI
No idea what went wrong here!
23 Gabon GA
No idea what went wrong here!
24 Italy IT
No idea what went wrong here!
25 Philippines PH
No idea what went wrong here!
26 USA US
27 USA US
No idea what went wrong here!
28 Sudan (South Sudan) SD
29 Zaire CD
30 England 

In [119]:
print len(all_locations["coordinates"])
print all_locations["coordinates"]

33
[(1.5333554, 32.2166578), (-2.9814344, 23.8222636), (1.5333554, 32.2166578), (1.5333554, 32.2166578), (-2.9814344, 23.8222636), (12.7503486, 122.7312101), (0, 0), (-2.9814344, 23.8222636), (64.6863136, 97.7453061), (7.8699431, 29.6667897), (-0.7264327, 15.6419155), (-0.7264327, 15.6419155), (-0.7264327, 15.6419155), (-0.8999695, 11.6899699), (1.5333554, 32.2166578), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (39.7837304, -100.4458825), (0, 0), (7.8699431, 29.6667897), (-2.9814344, 23.8222636), (52.7954791, -0.540240236617432), (7.8699431, 29.6667897), (-2.9814344, 23.8222636)]
